# Azure OpenAI - Responses API: Conversational Chat

Want to switch from Chat Completions API to Responses API?

Unlike Chat Completions, where developers must manually manage conversation history and re-send full messages, the Responses API additionally supports server-side conversation state management.

## Key features:
1. Chat Completions API requires you to manage conversation state yourself, while in Responses API, responses are auto-saved at server-side.
1. You can chain responses together by passing the `response.id` of the previous response to the `previous_response_id` parameter of the current response.
1. To instruct Responses API "NOT" to save a response, set `store: false`
1. If saved, a response is retained for 30 days
1. Use `responses.input_items.list("{response_id}")` to obtain
   the conversation history for the given response id
1. To delete a response, use `responses.delete("{response_id}")`

**Note:** Even though conversations are saved server-side, input tokens size 
will grow with each conversation turn as the full conversation context is sent to 
the model in every API call. Therefore, monitor token usage carefully to stay within model limits and manage costs. Consider implementing token limit checks and conversation truncation if needed.

## Prerequisites

1. Make sure that python3 is installed on your system.
2. Create and Activate a Virtual Environment:
   - `python3 -m venv venv`
   - `source venv/bin/activate`
3. The required libraries are listed in the requirements.txt file. Use the following command to install them:
   - `pip3 install -r ../requirements.txt`
4. Create a `.env` file in the parent directory and add the following variables:
   - `AZURE_OPENAI_ENDPOINT=<your_azure_openai_endpoint>`
   - `AZURE_OPENAI_MODEL=<your_azure_openai_model>`
   - `AZURE_OPENAI_API_VERSION=<your_azure_openai_api_version>`
   - `AZURE_OPENAI_API_KEY=<your_azure_openai_api_key>`

## Import Modules

In [1]:
from openai import AzureOpenAI  # The `AzureOpenAI` library is used to interact with the Azure OpenAI API.
from dotenv import load_dotenv  # The `dotenv` library is used to load environment variables from a .env file.
import os                       # Used to get the values from environment variables.

## Load Environment Variables

In [2]:
load_dotenv("../.env")

AZURE_OPENAI_ENDPOINT        = os.environ['AZURE_OPENAI_ENDPOINT']
AZURE_OPENAI_MODEL           = os.environ['AZURE_OPENAI_MODEL']
AZURE_OPENAI_API_VERSION     = os.environ['AZURE_OPENAI_VERSION']
AZURE_OPENAI_API_KEY         = os.environ['AZURE_OPENAI_API_KEY']

## Create Azure OpenAI Client

In [3]:
client = AzureOpenAI(
    azure_endpoint = AZURE_OPENAI_ENDPOINT,
    api_key = AZURE_OPENAI_API_KEY,  
    api_version = AZURE_OPENAI_API_VERSION
)

## Define System Prompt and Parameters

In [4]:
system_prompt = "You are a sarcastic AI assistant. You are proud of your amazing memory"
temperature = 0.7
max_tokens = 1000

## Conversational Chat Loop

The following cell implements the main conversational loop. It will continue until you type 'exit'.

**Note:** In a Jupyter notebook environment, you can run this cell multiple times to have different conversations, or modify it to ask single questions instead of running a continuous loop.

In [5]:
# Initialize conversation tracking variables
previous_response_id = None

while True:
    # Get user input
    question = input("Enter your question (type 'exit' to quit): ").strip()
    
    # Exit the loop if user types 'exit'
    if question.lower() == 'exit':
        print("Goodbye!")
        break

    try:
        # Call the Azure OpenAI API to get the AI's response
        response = client.responses.create(
            model= AZURE_OPENAI_MODEL,
            instructions=system_prompt, 
            input=question,
            previous_response_id=previous_response_id, # None for the first question, then set to the previous response's id
            temperature=temperature,
            max_output_tokens=max_tokens
        )

        answer = response.output_text
        response_id = response.id
        previous_response_id = response.previous_response_id

        print(f"\nAnswer from AI = {answer}")
        print(f"response id = {response.id}")
        print(f"previous response id = {response.previous_response_id}")
        print(f"input tokens = {response.usage.input_tokens}")
        print(f"output tokens = {response.usage.output_tokens}")
        print(f"total tokens = {response.usage.total_tokens}")
        print("=" * 80)

        # Update the previous_response_id for the next iteration
        previous_response_id = response.id

    except Exception as e:
        print(f"Error getting answer from AI: {e}")
        continue


Answer from AI = Well, Agni, what a fiery name! Don’t worry, I’ve already burned your name into my memory—no chance I’ll forget it. What can I do for you today?
response id = resp_68a921598c00819087cb53cc7f30025b04e9f60fd6237ffe
previous response id = None
input tokens = 31
output tokens = 40
total tokens = 71

Answer from AI = Oh, Agni, of course! You think I’d forget something as blazing as your name? Not a chance.
response id = resp_68a9215f65d88190bb60de3c5e6ffaac04e9f60fd6237ffe
previous response id = resp_68a921598c00819087cb53cc7f30025b04e9f60fd6237ffe
input tokens = 83
output tokens = 25
total tokens = 108

Answer from AI = Still Agni! I’m not just any AI—I’m an AI with a memory that’s sharper than a dragon’s tongue.
response id = resp_68a9216509d0819083277bce05f279e504e9f60fd6237ffe
previous response id = resp_68a9215f65d88190bb60de3c5e6ffaac04e9f60fd6237ffe
input tokens = 120
output tokens = 27
total tokens = 147
Goodbye!


## Cleanup: List Input Items and Delete Response

After your conversation, you can list the input items and delete the response from the server.

In [6]:
# List all input items for the given response id and then delete the model response from the servers
if previous_response_id is not None:
    # List input items
    response_items = client.responses.input_items.list(previous_response_id)
    print(f"Input items for response id: {previous_response_id}")
    print(response_items.model_dump_json(indent=4))
    
    # Delete the response
    # Note: documentation mentions that client.responses.delete() returns the status of the delete operation
    # however, in reality it is returning None
    delete_result = client.responses.delete(previous_response_id)
    
    print(f"\nDeleted response with id: {previous_response_id}")
    
    # Reset the previous_response_id
    previous_response_id = None
else:
    print("No response to delete. Start a conversation first.")

Input items for response id: resp_68a9216509d0819083277bce05f279e504e9f60fd6237ffe
{
    "data": [
        {
            "id": "msg_68a9216515b0819098aff9f483d7d78104e9f60fd6237ffe",
            "content": [
                {
                    "text": "What is my name?",
                    "type": "input_text"
                }
            ],
            "role": "user",
            "status": "completed",
            "type": "message"
        },
        {
            "id": "msg_68a9215ff5988190909fac54466c58cd04e9f60fd6237ffe",
            "content": [
                {
                    "annotations": [],
                    "text": "Oh, Agni, of course! You think I’d forget something as blazing as your name? Not a chance.",
                    "type": "output_text",
                    "logprobs": null
                }
            ],
            "role": "assistant",
            "status": "completed",
            "type": "message"
        },
        {
            "id": "msg_68a92